<a href="https://colab.research.google.com/github/StevenWestmoreland/DS-Unit-2-Linear-Models/blob/master/DSPT6_U2_S1_Regression2Assignment_StevenWestmoreland.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [110]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# create a column where 'created' is a datetime dtype
df['datetime_created']=pd.to_datetime(df['created'], infer_datetime_format=True)

In [0]:
# create a month column for easy train/test separation
df['month_created']=df['datetime_created'].dt.month

In [139]:
# set train data to April and May
train = df[df['month_created'] <= 5]
train

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,datetime_created,month_created,perk_number,bed_bath,interest_value
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-04-17 03:26:41,4,3,2.0,3
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-04-18 02:22:02,4,2,2.0,1
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-04-28 01:32:41,4,1,5.0,1
5,2.0,4,2016-04-19 04:24:47,,West 18th Street,40.7429,-74.0028,7995,350 West 18th Street,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-04-19 04:24:47,4,0,6.0,2
6,1.0,2,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,40.8012,-73.9660,3600,210 West 107th Street,low,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-04-27 03:19:56,4,3,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49346,1.0,1,2016-04-22 15:44:11,24hr Doorman Luxury building in the heart of t...,East 10th Street,40.7296,-73.9869,4500,166 2nd avenue,medium,1,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-04-22 15:44:11,4,5,2.0,2
49348,1.0,1,2016-04-04 18:22:34,"HIGH END condo finishes, swimming pool, and ki...",Rector Pl,40.7102,-74.0163,3950,225 Rector Place,low,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,2016-04-04 18:22:34,4,9,2.0,1
49349,1.0,1,2016-04-16 02:13:40,Large Renovated One Bedroom Apartment with Sta...,West 45th Street,40.7601,-73.9900,2595,341 West 45th Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-04-16 02:13:40,4,5,2.0,1
49350,1.0,0,2016-04-08 02:13:33,Stylishly sleek studio apartment with unsurpas...,Wall Street,40.7066,-74.0101,3350,37 Wall Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-04-08 02:13:33,4,5,1.0,1


In [140]:
# set test data to June
test = df[df['month_created'] > 5]
test

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,datetime_created,month_created,perk_number,bed_bath,interest_value
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-24 07:54:24,6,0,4.5,2
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-12 12:19:27,6,5,3.0,1
11,1.0,1,2016-06-03 03:21:22,Check out this one bedroom apartment in a grea...,W. 173rd Street,40.8448,-73.9396,1675,644 W. 173rd Street,low,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-03 03:21:22,6,0,2.0,1
14,1.0,1,2016-06-01 03:11:01,Spacious 1-Bedroom to fit King-sized bed comfo...,East 56th St..,40.7584,-73.9648,3050,315 East 56th St..,low,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-01 03:11:01,6,3,2.0,1
24,2.0,4,2016-06-07 04:39:56,SPRAWLING 2 BEDROOM FOUND! ENJOY THE LUXURY OF...,W 18 St.,40.7391,-73.9936,7400,30 W 18 St.,medium,1,1,1,1,1,1,0,0,1,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,2016-06-07 04:39:56,6,11,6.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49305,1.0,2,2016-06-16 04:20:46,Spacious sunny 2 bedroom apartment/ Queen size...,W 175 Street,40.8456,-73.9361,2295,575 W 175 Street,low,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-16 04:20:46,6,2,3.0,1
49310,1.0,3,2016-06-21 06:25:35,"Soaring 40 stories into the Manhattan skyline,...",Second Avenue,40.7817,-73.9497,3995,1751 Second Avenue,high,1,0,1,0,1,1,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2016-06-21 06:25:35,6,8,4.0,3
49320,1.0,1,2016-06-02 13:24:18,Great deal for a one bedroom located in Prime ...,West 55th Street,40.7669,-73.9917,2727,448 West 55th Street,medium,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-02 13:24:18,6,2,2.0,2
49332,1.0,2,2016-06-06 01:22:44,Don't miss out on this spacious and beautiful ...,West 98th Street,40.7957,-73.9705,4850,220 West 98th Street,low,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-06 01:22:44,6,4,3.0,1


In [115]:
train.shape, test.shape

((31844, 36), (16973, 36))

### Baseline (0 features)

In [116]:
# find average price of apartments in April and May
train['price'].mean()

3575.604007034292

In [0]:
# arrange y target vectors
target = 'price'
y_train = train[target]
y_test = test[target]

In [118]:
# get a mean baseline
print('Mean Baseline for 0 features')
guess=y_train.mean()
guess

Mean Baseline for 0 features


3575.604007034292

In [119]:
# train error
from sklearn.metrics import mean_absolute_error
y_pred=[guess] * len(y_train)
mae=mean_absolute_error(y_train,y_pred)
print(f'Train Error (March-April Apartment Prices: ${mae:.2f}')

Train Error (March-April Apartment Prices: $1201.88


In [120]:
# test error
y_pred=[guess] * len(y_test)
mae=mean_absolute_error(y_test,y_pred)
print(f'Train Error (March-April Apartment Prices: ${mae:.2f}')

Train Error (March-April Apartment Prices: $1197.71


### Two-feature Regression using number of perks and combined number of bedrooms and bathrooms.

In [121]:
# create columns with new information
df['perk_number']=(df['elevator']+df['cats_allowed']+df['hardwood_floors']
                   +df['dogs_allowed']+df['doorman']+df['dishwasher']+df['no_fee']
                   +df['laundry_in_building']+df['fitness_center']+df['pre-war']
                   +df['laundry_in_unit']+df['roof_deck']+df['outdoor_space']
                   +df['dining_room']+df['high_speed_internet']+df['balcony']
                   +df['swimming_pool']+df['new_construction']+df['terrace']
                   +df['exclusive']+df['loft']+df['garden_patio']
                   +df['wheelchair_access']+df['common_outdoor_space'])
df['bed_bath']=df['bedrooms']+df['bathrooms']
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,datetime_created,month_created,perk_number,bed_bath
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-24 07:54:24,6,0,4.5
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-06-12 12:19:27,6,5,3.0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-04-17 03:26:41,4,3,2.0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-04-18 02:22:02,4,2,2.0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-04-28 01:32:41,4,1,5.0


In [125]:
# Arrange X features matrices
features=['bed_bath','perk_number']
print(f'Linear Regression, dependent on: {features}')

X_train=train[features]
X_test=test[features]

Linear Regression, dependent on: ['bed_bath', 'perk_number']


In [126]:
X_train.shape, X_test.shape

((31844, 2), (16973, 2))

In [0]:
# import and instantiate LinearRegression()
from sklearn.linear_model import LinearRegression
model=LinearRegression()

In [128]:
# fit the model
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [129]:
y_pred_train=model.predict(X_train)
y_pred_train

array([2839.4009342 , 2737.63658265, 4939.74209926, ..., 3042.92963729,
       2274.97301457, 3403.82885382])

In [135]:
# find MAE
y_pred_test=model.predict(X_test)
print('Mean Absolute Error:',mean_absolute_error(y_test,y_pred_test))

Mean Absolute Error: 867.9967359241994


In [131]:
# find coefficient and intercept

print('Intercept', model.intercept_)
coefficients = pd.Series(model.coef_, features)
print(coefficients.to_string())

Intercept 998.1946341143803
bed_bath       767.956623
perk_number    101.764352


In [136]:
# find RMSE, and R^2
mse=mean_squared_error(y_test,y_pred_test)
rmse=np.sqrt(mse)
r2=r2_score(y_test,y_pred_test)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('R^2:',r2)

Mean Squared Error: 1672432.7152729623
Root Mean Squared Error: 1293.2257015977382
R^2: 0.46189791795456303


### Three-feature Regression, adding in interest level 

In [137]:
# chack value counts to see how many bins to make
df['interest_level'].value_counts()

low       33946
medium    11181
high       3690
Name: interest_level, dtype: int64

In [0]:
# map low, medium, and high to numerical values
df['interest_value']=df['interest_level'].map({'low':1,'medium':2,'high':3})

In [141]:
# update features
features2=['bed_bath','perk_number','interest_value']
print(f'Linear Regression, dependent on: {features2}')

X_train2=train[features2]
X_test2=test[features2]

Linear Regression, dependent on: ['bed_bath', 'perk_number', 'interest_value']


In [142]:
# fit the model
model.fit(X_train2, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [143]:
y_pred_train2=model.predict(X_train2)
y_pred_train2

array([1859.74141736, 2956.54980909, 5152.27813011, ..., 3275.1571503 ,
       2507.84689427, 3617.65761805])

In [144]:
# find MAE
y_pred_test2=model.predict(X_test2)
mean_absolute_error(y_test,y_pred_test2)

818.3364084030351

In [148]:
# find coefficient and intercept

print('Intercept', model.intercept_)
coefficients = pd.Series(model.coef_, features2)
print(coefficients.to_string())

Intercept 1811.0298222870085
bed_bath          767.310256
perk_number       106.202447
interest_value   -601.505419


In [147]:
# find RMSE, and R^2
mse=mean_squared_error(y_test,y_pred_test2)
rmse=np.sqrt(mse)
r2=r2_score(y_test,y_pred_test2)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('R^2:',r2)

Mean Squared Error: 1515608.6420363358
Root Mean Squared Error: 1231.1005816083168
R^2: 0.5123557686966791
